In [190]:
# 요기요 크롤링
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By # 원하는 부분을 추출할 수 있는 방식을 지정하는 객체
from pandas import DataFrame
import openpyxl


In [191]:

# url입력
driver = webdriver.Chrome()
driver.get('https://www.yogiyo.co.kr/mobile/#/')
time.sleep(2) # 2초 지연


In [192]:
# 검색창 선택
xpath = '''//*[@id="search"]/div/form/input''' # 검색창
element = driver.find_element(By.XPATH,xpath)
element.clear()
time.sleep(2)

In [193]:
# 검색창 입력
# 검색 지역 
location = '서울특별시 서초구 방배동 836-10 삼경빌딩'
element.send_keys(location)
element.send_keys(Keys.RETURN)
time.sleep(1)


In [194]:
# # 반복해서 가져올 url
category = ['1인분주문','프랜차이즈','치킨','피자양식','중식','한식','일식돈까스','족발보쌈','야식','분식','카페디저트','편의점']


# 수집된 결과가 누적될 빈 리스트
yogiyoData = []

# 카테고리별 반복 
for c in category :

    # 카테고리로 이동한다. 
    driver.get('https://www.yogiyo.co.kr/mobile/#/{}'.format(c))
    time.sleep(2) 

    # 스크롤 반복해서 끝까지 내리기 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # 스크롤을 가장 아래로 내린다
    time.sleep(3)
    pre_height = int(driver.execute_script("return document.body.scrollHeight")) # 현재 스크롤 위치 저장
    while True :
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")  # 스크롤을 가장 아래로 내린다
        cur_height = int(driver.execute_script("return document.body.scrollHeight"))  # 현재 스크롤을 저장한다.
        if pre_height == cur_height :
            break
        pre_height == cur_height
    time.sleep(3)

    
    # 가게 정보가 나오는 부분만 추출하기
    soup = BeautifulSoup(driver.page_source)
    storeList = soup.select('.col-sm-6') 
    
    # 가게별 정보 가져오기 
    for s in storeList :

        # 매장명 가져오기 (# storeName = s.select('.restaurants-info > .restaurant-name ng-binding'))       
        name = s.find_all("div", class_= "restaurant-name ng-binding")
        if name :
            store_name = name[0].text.strip()
        else :
            store_name = None 

        try : # 별점/리뷰/사장님댓글쿠폰을 없을 수 있기 때문에 예외처리로 묶음 

            # 별점 가져오기 
            star = s.find_all("span", class_ = "ico-star1 ng-binding") 
            if star :
                store_star = float(star[0].text.strip().replace("★ ",""))

            # 리뷰 수 가져오기 
            review = s.find_all("span",class_ = "review_num ng-binding")
            if review[0] :
                store_review = int(review[0].text.strip().replace("리뷰 ",""))

            # 사장님 댓글 수 가져오기 
            if review[1] :
                ceo_review = int(review[1].text.strip().replace("사장님댓글 ",""))

            # 쿠폰 
            coupon = s.find_all("span",class_ = "coupon-base ng-binding")
            if coupon :
                store_coupon = coupon[0].text.strip()

        except : # 없어서 에러가 나는 경우 
            store_star = None
            store_review = None
            ceo_review = None
            store_coupon = '행사없음'


            # 결과 병합
        resultDic = {'카테고리':c,'매장명':store_name,"별점":store_star,'리뷰':store_review,'사장님댓글':ceo_review,'쿠폰':store_coupon}
        yogiyoData.append(resultDic)

df = DataFrame(yogiyoData)
df.to_excel('요기요 (%s).xlsx' % location,sheet_name='요기요',index=False)
df

,카데고리,매장명,별점,리뷰,사장님댓글,쿠폰
0,1인분주문,큰맘할매순대국-이수역점,4.6,523.0,9.0,"2,000원 할인"
1,1인분주문,맥도날드-방배점,NaN,NaN,NaN,행사없음
2,1인분주문,정육식당-미트온라인직판,4.9,932.0,848.0,행사없음
3,1인분주문,롯데리아-남성역점,4.9,1944.0,1763.0,행사없음
4,1인분주문,맛없으면환불해주는김치찜-서초점,5.0,7.0,7.0,행사없음
...,...,...,...,...,...,...
1265,카페디저트,파리바게뜨-방배현대점,4.9,763.0,489.0,"3,000원 할인"
1266,카페디저트,이삭토스트-총신대점,NaN,NaN,NaN,행사없음
1267,카페디저트,일단시켜찹쌀타코야끼&카페,NaN,NaN,NaN,행사없음
1268,카페디저트,사당1038,NaN,NaN,NaN,행사없음
